In [ ]:
# To use group convolutions, clone https://github.com/tscohen/GrouPy and run
# python setup.py install
# (inside your virtualenv if you are using one)



In [1]:
import numpy as np
import tensorflow as tf
from groupy.gconv.tensorflow_gconv.splitgconv2d import gconv2d, gconv2d_util

# Construct graph
input_shape=(20,20,8)
x = tf.placeholder(tf.float32, [None, *input_shape])



w = tf.Variable(tf.truncated_normal([5, 5, input_shape[2], 32], stddev=0.1), name="W")
b = tf.Variable(tf.constant(0.1, shape=[32]), name="B")
y1 = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding="SAME")
    
gconv_indices, gconv_shape_info, w_shape = gconv2d_util(
    h_input='Z2', h_output='D4', in_channels=32, out_channels=64, ksize=3)
w = tf.Variable(tf.truncated_normal(w_shape, stddev=1.))
y2 = gconv2d(input=y1, filter=w, strides=[1, 2, 2, 1], padding='SAME',
            gconv_indices=gconv_indices, gconv_shape_info=gconv_shape_info)


gconv_indices, gconv_shape_info, w_shape = gconv2d_util(
    h_input='D4', h_output='D4', in_channels=64, out_channels=64, ksize=3)
w = tf.Variable(tf.truncated_normal(w_shape, stddev=1.))
y = gconv2d(input=y2, filter=w, strides=[1, 1, 1, 1], padding='SAME',
            gconv_indices=gconv_indices, gconv_shape_info=gconv_shape_info)

# Compute
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
yval = sess.run(y, feed_dict={x: np.random.randn(10, *input_shape)})
sess.close()

print(yval.shape)  # (10, 9, 9, 512) 

/home/facuq/dev/exp/rotation/.env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(10, 10, 10, 512)


In [2]:

import tensorflow as tf
sess = tf.Session()

from keras import backend as K
from keras.layers import Dense, Flatten,Conv2D
K.set_session(sess)

from keras.models import Sequential
from layers import GConv2D

input_shape=(9,9,3)
batch_size=10
model = Sequential()

model.add(GConv2D(out_channels=32,h_input='Z2',h_output='C4',input_shape=input_shape))
model.add(GConv2D(out_channels=64,h_input='C4',h_output='C4'))
model.add(GConv2D(out_channels=64,h_input='C4',h_output='C4'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

# this works! 

x = tf.placeholder(tf.float32, shape=(None, *input_shape))
y = model(x)

init_op = tf.global_variables_initializer()
sess.run(init_op)
npx=np.random.rand(10,*input_shape)
y.eval(feed_dict={x: npx},session=sess)



Using TensorFlow backend.


array([[0.10209556, 0.08754663, 0.11159792, 0.09038323, 0.08696858,
        0.09631675, 0.10398663, 0.14031078, 0.09905198, 0.08174188],
       [0.10704695, 0.0861169 , 0.1009955 , 0.09479278, 0.09143273,
        0.10204962, 0.11257763, 0.11909661, 0.11133212, 0.07455917],
       [0.10039984, 0.09669842, 0.10071056, 0.08946796, 0.09301144,
        0.10406072, 0.09987837, 0.13621002, 0.0958347 , 0.0837279 ],
       [0.10230449, 0.0870057 , 0.1024095 , 0.0985565 , 0.0929886 ,
        0.08675032, 0.1093635 , 0.13193783, 0.10424849, 0.08443514],
       [0.09696353, 0.08728697, 0.10003261, 0.10147282, 0.09814094,
        0.10090251, 0.10016499, 0.13022289, 0.10355845, 0.08125439],
       [0.09305371, 0.09504005, 0.09875611, 0.09583359, 0.09760455,
        0.09966935, 0.10246343, 0.13206959, 0.10671861, 0.07879101],
       [0.09190694, 0.10179716, 0.10632059, 0.10436914, 0.10606608,
        0.09339731, 0.10710761, 0.1201153 , 0.09038597, 0.07853398],
       [0.10569593, 0.09609272, 0.0947135